In [17]:
import shutil
import os

source_path = "/kaggle/input/datasets/yuvrajsingh14/crack-or-noncrack"
destination_path = "/kaggle/working/dataset"

shutil.copytree(source_path, destination_path)

print("Dataset copied to working directory ")

Dataset copied to working directory ✅


In [18]:
train_path = "/kaggle/working/dataset/train"

In [19]:
import os
import shutil

train_path = "/kaggle/working/dataset/train"

for file in os.listdir(train_path):
    if not file.endswith(".png"):
        continue

    number = int(file.split(".")[0])
    src = os.path.join(train_path, file)

    if 1 <= number <= 3068:
        dst_folder = "/kaggle/working/dataset/train/defect"
    elif 3069 <= number <= 6138:
        dst_folder = "/kaggle/working/dataset/train/no_defect"
    elif 6139 <= number <= 14138:
        dst_folder = "/kaggle/working/dataset/train/defect"
    elif 14139 <= number <= 22112:
        dst_folder = "/kaggle/working/dataset/train/no_defect"
    else:
        continue

    os.makedirs(dst_folder, exist_ok=True)
    shutil.move(src, os.path.join(dst_folder, file))

print("Sorting Complete ✅")

Sorting Complete ✅


In [20]:
train_dataset = datasets.ImageFolder("/kaggle/working/dataset/train", transform=train_transform)
val_dataset = datasets.ImageFolder("/kaggle/working/dataset/val", transform=val_transform)

In [21]:
print(len(os.listdir("/kaggle/working/dataset/train/defect")))
print(len(os.listdir("/kaggle/working/dataset/train/no_defect")))

11068
11044


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# Strong Augmentation (important for generalization)
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder("/kaggle/working/dataset/train", transform=train_transform)
val_dataset = datasets.ImageFolder("/kaggle/working/dataset/val", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# EfficientNet
model = models.efficientnet_b0(weights="IMAGENET1K_V1")
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

best_acc = 0

for epoch in range(15):
    model.train()
    total_loss = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    
    val_acc = correct / total
    print(f"Epoch {epoch+1}: Val Accuracy = {val_acc:.4f}")
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "/kaggle/working/best_model.pth")
        print("✅ Best model saved")

print("🔥 Best Validation Accuracy:", best_acc)

Using: cuda
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 129MB/s] 


Epoch 1: Val Accuracy = 0.6449
✅ Best model saved
Epoch 2: Val Accuracy = 0.6287
Epoch 3: Val Accuracy = 0.6503
✅ Best model saved
Epoch 4: Val Accuracy = 0.6061
Epoch 5: Val Accuracy = 0.6768
✅ Best model saved
Epoch 6: Val Accuracy = 0.6509
Epoch 7: Val Accuracy = 0.7634
✅ Best model saved
Epoch 8: Val Accuracy = 0.6293
Epoch 9: Val Accuracy = 0.5914
Epoch 10: Val Accuracy = 0.7018
Epoch 11: Val Accuracy = 0.7063
Epoch 12: Val Accuracy = 0.6153
Epoch 13: Val Accuracy = 0.5845
Epoch 14: Val Accuracy = 0.6801
Epoch 15: Val Accuracy = 0.6190
🔥 Best Validation Accuracy: 0.7633915359573838
